In [ ]:
import os
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple

from exp_spec_info import *
from plot_info import *
from select_data import *

In [ ]:
# Processed median path
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

# Matrix data
small_matrix_data_path = "C:\\Users\\dosre\\dev\\precision-cascade\\scripts\\python\\view_experimentation\\data\\small-experimental-matrix-data.csv"
large_matrix_data_path = "C:\\Users\\dosre\\dev\\precision-cascade\\scripts\\python\\view_experimentation\\data\\large-experimental-matrix-data.csv"

# Plot output directory
chap_6_image_dir = "C:\\Users\\dosre\\Desktop\\MSCthesis\\thesis\\body_chapters\\chap_6\\images"
os.makedirs(chap_6_image_dir, exist_ok=True)
unprecond_dir = os.path.join(chap_6_image_dir, "unprecond-analysis")
os.makedirs(unprecond_dir, exist_ok=True)
precond_dir = os.path.join(chap_6_image_dir, "precond-analysis")
os.makedirs(precond_dir, exist_ok=True)

In [ ]:
# Load matrix data
small_matrix_data = pd.read_csv(small_matrix_data_path).rename({"name": "matrix"}, axis=1)
large_matrix_data = pd.read_csv(large_matrix_data_path).rename({"name": "matrix"}, axis=1)
all_matrix_data = pd.concat([small_matrix_data.drop("cond", axis=1), large_matrix_data])

# Load data and filter for only gmres m solvers
median_data = pd.read_pickle(median_data_path)
unprecond_median_data = median_data[
    (median_data["solver"].transform(lambda x: (x in PC_SOLVERS))) &
    median_data["setup"].isin({"unprecond", "unpreconddense"})
]
precond_median_data = median_data[
    (median_data["solver"].transform(lambda x: (x in PC_SOLVERS))) &
    median_data["setup"].isin({"ilu0", "ilutp1em2", "ilutp1em4"})
]

##### 6.3/6.4 Prune Non-Complaint Experiments

In [ ]:
import matplotlib.gridspec


def prune_analyze_non_compliant_relres_data(
    data: pd.DataFrame, threshold: float, plot_title: str, plot_save_path: str
) -> pd.DataFrame:

    failed_fp64_exp = (
        (data["med_rel_res"]/(data["med_rel_res_frac_err_fp64"] + 1)) > 1
    )
    keep = ~failed_fp64_exp & (data["med_rel_res_frac_err_fp64"] <= threshold)
    compliant_data = data[keep]
    eliminated_data = data[~keep]

    elim_mat = eliminated_data["matrix"].unique()
    n_elim_mat = len(elim_mat)
    mat_index_dict = dict(zip(elim_mat, range(n_elim_mat)))
    index_mat_dict = dict(zip(range(n_elim_mat), elim_mat))

    solver_percents = (eliminated_data["solver"].value_counts()/len(eliminated_data)).round(2)
    restart_param_percents = ((eliminated_data["restart_param"].value_counts()/len(eliminated_data)).round(2))

    fig = plt.figure(figsize=(6.4, 5.6))
    spec = matplotlib.gridspec.GridSpec(
        ncols=2,
        nrows=3,
        width_ratios=[0.6, 0.4],
        height_ratios=[0.36, 0.38, 0.25],
        hspace=0.05,
        wspace=0.05,
        figure=fig
    )
    ax = fig.add_subplot(spec[:, 0])
    ax1 = fig.add_subplot(spec[0, 1])
    ax2 = fig.add_subplot(spec[1, 1])
    ax3 = fig.add_subplot(spec[2, 1])

    for solver in PC_SOLVERS:
        select = eliminated_data["solver"] == solver
        ax.scatter(
            eliminated_data["med_rel_res_frac_err_fp64"][select],
            eliminated_data["matrix"][select].transform(
                lambda mat: mat_index_dict[mat]
            ),
            eliminated_data["restart_param"][select],
            marker=".",
            color=SOLVER_CLR_DICT[solver],
            label=solver
        )

    ax.grid()
    ax.set_axisbelow(True)

    ax1.legend(
        handles=[matplotlib.lines.Line2D([0], [0], marker="o", linestyle="None", color=SOLVER_CLR_DICT[solver]) for solver in PC_SOLVERS],
        labels=[f"{solver} ({solver_percents[solver]:.2f})" for solver in PC_SOLVERS],
        loc="upper left",
        bbox_to_anchor = (0., 1.),
        borderaxespad=0.
    )
    ax2.legend(
        handles=[matplotlib.lines.Line2D([0], [0], marker=".", markersize=np.sqrt(val), linestyle="None", color="grey") for val in RESTART_PARAMS],
        labels=[f"$m$={val} ({restart_param_percents[val]:.2f})" for val in RESTART_PARAMS],
        loc="upper left",
        bbox_to_anchor = (0., 1.),
        borderaxespad=0.
    )
    
    elim_data_frac = len(eliminated_data)/len(data)
    table = ax3.table(
        [
            ["Eliminated Data Frac."],
            [f"{elim_data_frac:.2f}"],
            ["FP64 Failed Frac."],
            [f"{failed_fp64_exp.sum()/len(eliminated_data):.2f}"]
        ],
        bbox=(0., 0., .9, .7)
    )
    table.auto_set_font_size(False)
    for info_ax in [ax1, ax2, ax3]:
        info_ax.tick_params(
            axis="both", bottom=False, left=False, labelbottom=False, labelleft=False
        )
        info_ax.axis("off")

    ax.semilogx()
    ax.set_xlabel("Med. Rel. Res. Frac. Error to FP64")
    ax.set_yticks(range(n_elim_mat))
    ax.set_yticklabels([index_mat_dict[i] for i in range(n_elim_mat)])
    ax.set_ylabel("Linear Solve Matrix")
    ax.set_title(plot_title)

    plt.savefig(plot_save_path, bbox_inches="tight")
    plt.show()
    plt.close()

    return compliant_data

unprecond_compliant_data = prune_analyze_non_compliant_relres_data(
    unprecond_median_data,
    1.,
    f"Non-Compliant Unpreconditioned Experiments with Rel. Res. Frac. Error 1-Threshold",
    os.path.join(unprecond_dir, "unprecond-threshold-non-compliance.png")
)

precond_compliant_data = prune_analyze_non_compliant_relres_data(
    precond_median_data,
    1.,
    f"Non-Compliant Preconditioned Experiments with Rel. Res. Frac. Error 1-Threshold",
    os.path.join(precond_dir, "precond-threshold-non-compliance.png")
)

##### 6.3/6.4 Unpreconditioned FP64 FP-GMRES($m$) and PC-GMRES($m$) Relative Time Comparison

In [ ]:
def analyze_rel_time(
    data: pd.DataFrame,
    setups: List[str],
    setup_formats: List[Tuple[str, str]],
    x_range: Tuple[float, float],
    plot_title_prefix: str,
    plot_dir: str,
    plot_save_file_title_prefix: str,
):

    matrices = data["matrix"].unique()
    n_matrices = len(matrices)
    mat_index_dict = dict(zip(matrices, range(n_matrices)))
    index_mat_dict = dict(zip(range(n_matrices), matrices))

    for restart_param in RESTART_PARAMS:

        restart_data = data[data["restart_param"] == restart_param]

        fig, axs = plt.subplots(
            1, len(PC_SOLVERS), figsize=(11, 8), sharey=True
        )

        for ax, solver in zip(axs, PC_SOLVERS):

            solver_data = restart_data[restart_data["solver"] == solver]

            for setup, setup_format in zip(setups, setup_formats):

                marker, line = setup_format

                current_data = solver_data[solver_data["setup"] == setup]

                ax.scatter(
                    current_data["med_rel_time_fp64"],
                    current_data["matrix"].transform(lambda mat: mat_index_dict[mat]),
                    color=SOLVER_CLR_DICT[solver],
                    marker=marker,
                    zorder=2
                )
                ax.axvline(
                    current_data["med_rel_time_fp64"].median(),
                    color=SOLVER_CLR_DICT[solver],
                    linestyle=line,
                    zorder=2
                )
                ax.axvline(1., color=SOLVER_CLR_DICT["FP FP64"], zorder=1)
                ax.set_title(f"{solver}")

                ax.set_xlim(x_range[0], x_range[1])

            rel_solve_data = solver_data[solver_data["setup"].isin(setups)]
            off_screen_data = (
                rel_solve_data[rel_solve_data["med_rel_time_fp64"] >= x_range[1]]["matrix"]
            ).value_counts()
            for matrix in off_screen_data.index:
                ax.text(
                    x_range[0]+0.99*(x_range[1]-x_range[0]),
                    mat_index_dict[matrix],
                    f"{off_screen_data.loc[matrix]}$\\rightarrow$",
                    verticalalignment='center',
                    horizontalalignment='right'
                )

        for ax in axs:
            ax.grid()
        axs[3].set_xlabel(f"Median Time Relative to FP64")
        axs[0].set_ylim(-0.75, n_matrices-0.25)
        axs[0].set_yticks(range(n_matrices))
        axs[0].set_yticklabels([index_mat_dict[i] for i in range(n_matrices)])
        axs[0].set_ylabel("Linear Solve Matrix")

        fig.suptitle(plot_title_prefix + f"PC-GMRES(${restart_param}$) Time Relative to FP64")
        fig.tight_layout()

        plt.savefig(
            os.path.join(
                plot_dir,
                f"{plot_save_file_title_prefix}-{restart_param:03d}.png"
            )
        )
        plt.show()
        plt.close()

analyze_rel_time(
    unprecond_compliant_data, ["unprecond", "unpreconddense"],
    [(".", ":"), ("x", "--")], (0., 3.), "Unpreconditioned ", unprecond_dir, "unprecond-rel-time"
)


In [ ]:
filt = unprecond_compliant_data[
    (unprecond_compliant_data["restart_param"] == 200) &
    (unprecond_compliant_data["med_rel_time_fp64"] >= 3.) &
    (unprecond_compliant_data["solver"] == "PC HSD ORC")
]

#### 6.3/6.4 PC-GMRES($m$) Comparison Matrix

In [ ]:
comp_data = pd.concat([unprecond_compliant_data, precond_compliant_data], axis=0)
comp_data = comp_data[["setup", "restart_param", "solver", "med_rel_time_fp64"]]
comp_data = comp_data.groupby(by=["setup", "restart_param", "solver"], as_index=False).median()
comp_data = comp_data.pivot(index=["setup", "solver"], columns="restart_param", values="med_rel_time_fp64")
comp_data = comp_data.round(2)

best_rel_time = comp_data.min().min()

In [ ]:
def pc_gmres_comp_matrix(data: pd.DataFrame, setup: str, cmap_range: Tuple[float, float], plot_save_title: str):

    setup_data = data.loc[setup].loc[PC_SOLVERS[::-1]]
    fig, ax = plt.subplots(figsize=(6.4, 4.8))

    solvers = setup_data.index.to_numpy()
    restart_params = setup_data.columns.to_numpy()

    X, Y = np.meshgrid(np.arange(0, len(restart_params)), np.arange(0, len(solvers)))
    quad_mesh = ax.pcolormesh(X, Y, setup_data.to_numpy(), cmap="RdYlBu_r", vmin=cmap_range[0], vmax=cmap_range[1])
    for i in np.arange(0, len(restart_params)):
        for j in np.arange(0, len(solvers)):
            ax.text(i, j, setup_data.loc[solvers[j]][restart_params[i]], ha="center", va="center", fontsize=9)

    ax.set_xticks(np.arange(0, len(restart_params)))
    ax.set_xticklabels([restart_params[i] for i in np.arange(0, len(restart_params))])
    ax.set_xlabel("Restart Parameter")
    ax.set_yticks(np.arange(0, len(solvers)))
    ax.set_yticklabels([solvers[i] for i in np.arange(0, len(solvers))])
    ax.set_ylabel("PC-GMRES($m$) Solver")
    ax.set_title(f"{SETUP_NAME_MAPPING[setup]} PC-GMRES($m$) Median Time Relative to FP64")

    cbar = plt.colorbar(quad_mesh, ax=ax)
    ticks = np.linspace(cmap_range[0], cmap_range[1], 5)
    cbar.ax.set_yticks(ticks)
    cbar.ax.set_yticklabels(
        [f"{tick:.2f}" for tick in ticks[:len(ticks)-1]] + [f"{cmap_range[1]:.2f}+"]
    )
    plt.savefig(plot_save_title, bbox_inches="tight")
    plt.show()

for setup, plot_dir in [
    ("unprecond", unprecond_dir),
    ("unpreconddense", unprecond_dir),
    ("ilu0", precond_dir),
    ("ilutp1em2", precond_dir),
    ("ilutp1em4", precond_dir)
]:
    pc_gmres_comp_matrix(
        comp_data,
        setup,
        (best_rel_time, 1.),
        os.path.join(plot_dir, f"rel-time-summary-{setup}.png")
    )

##### Misc. Analysis

In [ ]:
solver_data = median_data[
    (median_data["setup"] == "unprecond")
]

small_merge = solver_data.merge(small_matrix_data, on="matrix", how="inner")
large_merge = solver_data.merge(large_matrix_data, on="matrix", how="inner")

fig, ax = plt.subplots()
ax.scatter(small_merge["med_inner_iter"], small_merge["med_rel_time_fp64"], marker=".")
ax.scatter(large_merge["med_inner_iter"], large_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(
    np.hstack([small_merge["med_inner_iter"], large_merge["med_inner_iter"]]),
    np.hstack([small_merge["med_rel_time_fp64"], large_merge["med_rel_time_fp64"]])
))
plt.show()

fig, ax = plt.subplots()
ax.scatter(small_merge["nnz"], small_merge["med_rel_time_fp64"], marker=".")
ax.scatter(large_merge["nnz"], large_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(
    np.hstack([small_merge["nnz"], large_merge["nnz"]]),
    np.hstack([small_merge["med_rel_time_fp64"], large_merge["med_rel_time_fp64"]])
))
plt.show()

fig, ax = plt.subplots()
ax.scatter(small_merge["rows"], small_merge["med_rel_time_fp64"], marker=".")
ax.scatter(large_merge["rows"], large_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(
    np.hstack([small_merge["rows"], large_merge["rows"]]),
    np.hstack([small_merge["med_rel_time_fp64"], large_merge["med_rel_time_fp64"]])
))
plt.show()

fig, ax = plt.subplots()
ax.scatter(small_merge["cond"], small_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(small_merge["cond"], small_merge["med_rel_time_fp64"]))
plt.show()